In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Load model
model = load_model('pitchLine_model_safe.h5')

# Load and prepare dataset
df = pd.read_csv('output.csv')

input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets', 
              'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role', 
              'Batsman_Batting_Style', 'Batsman_Playing_Role']
output_col = 'pitchLine'

df = df.dropna(subset=input_cols + [output_col])

X = df[input_cols].copy()
y_true = df[output_col]

# Encode categorical variables
encoders = {}
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

# Encode output for comparison
output_encoder = LabelEncoder()
y_encoded = output_encoder.fit_transform(y_true)

# Predict (assume classification model with softmax output)
y_pred_probs = model.predict(X)
y_pred_encoded = np.argmax(y_pred_probs, axis=1)
y_pred_labels = output_encoder.inverse_transform(y_pred_encoded)

# Print 10 failure cases
failures = 0
for i in range(len(X)):
    if y_pred_labels[i] != y_true.iloc[i]:
        print(f"\nFailure case {failures + 1}:")
        print("Input:")
        for col in input_cols:
            print(f"  {col}: {df.iloc[i][col]}")
        print(f"Ground Truth: {y_true.iloc[i]}")
        print(f"Predicted   : {y_pred_labels[i]}")
        failures += 1
    if failures == 10:
        break


1614/1614 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step

Failure case 1:
Input:
  inningNumber: -0.748849161887193
  oversActual: -1.729534722737182
  totalRuns: -1.5684404745481182
  totalWickets: -1.2313666186679586
  time_of_day: -1.544842968766813
  Ground Name: -1.461343122078455
  Batsman_Name: 1.5578595784554623
  Batsman_Role: -0.1688643145826048
  Batsman_Batting_Style: 0.800225536895447
  Batsman_Playing_Role: 0.5627270004812331
Ground Truth: 1
Predicted   : 0

Failure case 2:
Input:
  inningNumber: -0.748849161887193
  oversActual: -1.7122192855241125
  totalRuns: -1.5684404745481182
  totalWickets: -1.2313666186679586
  time_of_day: -1.544842968766813
  Ground Name: -1.461343122078455
  Batsman_Name: 1.5578595784554623
  Batsman_Role: -0.1688643145826048
  Batsman_Batting_Style: 0.800225536895447
  Batsman_Playing_Role: 0.5627270004812331
Ground Truth: 1
Predicted   : 0

Failure case 3:
Input:
  inningNumber: -0.748849161887193
  oversActual: -1.6949038483110423
  totalRuns: -1.56844047